In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy.integrate import odeint, solve_ivp
final_fig_path = "../../Figures/"
from time import time

In [12]:
%%writefile fun_response_funs.py
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy.integrate import odeint, solve_ivp

    
def fun_response(x,M1,M2,index,a1,a2,h1,h2,**params):
    '''
    functional response to prey as a function of predator group size (x) and 
    prey population sizes (M1, M2)
    a_iα_i(x)M_i/(1 + h_1 a_1 α_1(x) M_1 + h_2 a_2 α_2(x) M_2)
    
    @inputs:
    x - pred group size
    M1 - big prey pop size
    M2 - small prey pop size
    index - 1 (big prey) or 2 (small prey)
    a1 - attack rate of big prey
    a2 - attack rate of small prey
    h1 - handling time per big prey caught
    h2 - handling time per small prey caught
    params: a dictionary of other parameters, that at least must include α1_of_1, α2_of_1, s1, s2

    @returns
    functional response for prey type <index> (a float)

    @examples
    >>fun_response(x=1,M1=10,M2=10,index=1,a1=1,a2=1,h1=0.5,h2=0.5, 
                    **dict(α1_of_1 = 0.05, α2_of_1 = 0.95, s1 = 2, s2 = 2) )
    0.08333333333333336
    (answer should be 10*a1*α1_of_1/(1+h1*a1*α1_of_1*10 + h2*a2*α2_of_1*10) = 0.08333333333333333
    
    '''
    
    α1 = fun_attack_rate(x,1,**params)
    α2 = fun_attack_rate(x,2,**params)
    if index == 1:
        numerator = a1*α1*M1
    elif index == 2:
        numerator = a2*α2*M2
    denominator = 1 + a1*α1*h1*M1 + a2*α2*h2*M2
    return numerator/denominator
def fun_response_non_dim(x,N1,N2,index,a,H1,H2, **params):
    '''
    non-dimensionalizedfunctional response to prey as a function of predator group size (x) and 
    non-dimensionalized versions of prey population sizes (N1, N2)
    a_iα_i(x)M_i/(1 + h_1 a_1 α_1(x) M_1 + h_2 a_2 α_2(x) M_2)
    
    @inputs:
    x - pred group size
    N1, N2 - big prey and small prey non-d. pop sizes (pop-sizes relative to carrying capacities, 
            so between 0 and 1)
    index - 1 (big prey) or 2 (small prey)
    a - ratio of attack rates, a_2/a_1
    H1, H2 - maximum time handling big prey or small prey, respectively
    params: a dictionary of other parameters, that at least must include α1_of_1, α2_of_1, s1, s2

    @returns
    functional response for prey type <index> (a float)

    @examples
    >>fun_response_non_dim(x=2,N1=0.5,N2=0.6,index=1,a=1,H1=2,H2=2, 
    >>                         **dict(α1_of_1 = 0.05, α2_of_1 = 0.95, s1 = 2, s2 = 2))
    0.11904761904761904 
    
    '''
    
    α1 = fun_attack_rate(x,1,**params)
    α2 = fun_attack_rate(x,2,**params)
    if index == 1:
        numerator = α1*N1
    elif index == 2:
        numerator = a*α2*N2
    denominator = 1 + α1*H1*N1 + α2*H2*N2
    return numerator/denominator


def fun_attack_rate(x, index, α1_of_1, α2_of_1, s1, s2, **params):
    '''
    The attack rate as a function of x
    
    @inputs:
    x: group size, 1,2,3,...
    index: 1 or 2, indicates prey type 1 (big prey) or 2 (small prey)
    α1_of_1: the attack rate of big prey for group size 1
    α2_of_1: the attack rate of small prey for group size 1
    s1: critical group size for big prey, must be >= 2
    s2: critical group size for small prey, must be >= 2
    
    @returns:
    attackrate (a float)

    @example:
    >> fun_attack_rate(1,2,0.05,0.95,2,2,**dict())
    0.9500000000000001
    >> fun_attack_rate(1,1,0.05,0.95,2,2,**dict())
    0.05000000000000001
    
    '''
    if index == 1:
        θ_1 = - np.log(1/α1_of_1 - 1)/(1-s1)
        return 1/(1 + np.exp(- θ_1 * (x - s1)))
    elif index == 2:
        θ_2 = - np.log(1/α2_of_1 - 1)/(1-s2)
        return 1/(1 + np.exp(- θ_2 * (x - s2)))
    

Overwriting fun_response_funs.py


In [6]:
%%writefile fitness_funs.py

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy.integrate import odeint, solve_ivp
from fun_response_funs import *


def fun_fitness(x, M1, M2, **params):
    '''
    this is a subordinate's fitness of being in a group of size x
    @inputs:
    x - grp size
    M1 - pop size of big prey
    M2 - pop size of small prey
    params - a dictionary at least including: b1, b2, r, γ, a1, a2, h1, h2, α1_of_1, 
                                                    α2_of_1, s1, s2, limited_portions
                                                    (and must also have b0 if limited_portions = True)
    @returns:
    fitness of a subordinate, a float (or array if one of the parameter inputs is an array)

    @example:
    >>fun_fitness(x=np.array([1,2,3]), M1=10, M2=10, **dict(b1=1,b2=0.1,r=0, γ=0, a1=1, a2=1, h1=0.5, h2=0.5, 
                                                    α1_of_1=0.05, α2_of_1=0.95, s1=2, s2=2, 
                                                    limited_portions = False))
    array([0.24166667, 0.45833333, 0.53055556])
    '''
    fitnesses = fitness_from_prey(x, M1, M2,**params)
    total_sub_fitness = np.sum(fitnesses,0)
    return total_sub_fitness



def per_capita_fitness_from_prey(x,M1,M2,b1,b2,limited_portions, **params):
    '''
    portion of direct fitness from each prey type without any skew, stored in an array
    @inputs:
    x - pred group size
    M1 - big prey pop size
    M2 - small prey pop size
    b1 - big prey  conversion (prey --> pred)
    b2 - small prey conversion (prey --> pred)
    limited_portions - True or False, whether predators can only eat a limited amount or not
    params - dictionary of other parameters, which must at least contain 
             a1, a2, h1, h2, α1_of_1, α2_of_1, s1, s2

    @returns:
    np.array([<inclusive fitness from big prey>, <inclusive fitness from small prey>])
    (so the rows correspond to prey types

    @example
    >>per_capita_fitness_from_prey(x= np.array([1,2,3]), M1=10, M2=10, **dict(b1=1,b2=0.1,r=0, γ=0, a1=1, a2=1, 
                                                        h1=0.5, h2=0.5, 
                                                    α1_of_1=0.05, α2_of_1=0.95, s1=2, s2=2, 
                                                    limited_portions = False))
    array([[0.08333333, 0.41666667, 0.52777778],
       [0.15833333, 0.04166667, 0.00277778]])
    
    >>per_capita_fitness_from_prey(x= np.array([1,2,3]), M1=10, M2=10, **dict(b1=1,b2=0.1,r=0, γ=0, a1=1, a2=1, 
                                                        h1=0.5, h2=0.5, 
                                                    α1_of_1=0.05, α2_of_1=0.95, s1=2, s2=2,b0 = 0.05, 
                                                    limited_portions = True))
    array([[0.00416667, 0.04166667, 0.07916667],
       [0.07916667, 0.04166667, 0.00277778]])
    '''
    # set portion size, need to account for x being an array
    conversion_big, conversion_small = conversion_prey(x,b1,b2,limited_portions,**params)

        
    w_per_capita = np.array([conversion_big*fun_response(x,M1,M2,1,**params), 
                         conversion_small*fun_response(x,M1,M2,2,**params)])
    return w_per_capita
    
def fitness_from_prey(x, M1, M2, r, γ,**params):
    '''
    portion of inclusive fitness from each prey type, stored in an array, after potentially unequal sharing
    @inputs:
    x - pred group size
    M1 - big prey pop size
    M2 - small prey pop size
    r - relatedness between group members
    γ - extent of reproductive skew (portion of subordinate's food donated to dominant)
    params - dictionary of other parameters, which must at least contain 
             a1, a2, h1, h2, α1_of_1, α2_of_1, s1, s2, b1, b2, limited_portions

    @returns:
    np.array([<inclusive fitness from big prey>, <inclusive fitness from small prey>])
    (so the rows correspond to prey types

    @example
    >>fitness_from_prey(x= np.array([1,2,3]), M1=10, M2=10, **dict(b1=1,b2=0.1,r=0, γ=0, a1=1, a2=1, 
                                                        h1=0.5, h2=0.5, 
                                                    α1_of_1=0.05, α2_of_1=0.95, s1=2, s2=2, 
                                                    limited_portions = False))
    array([[0.08333333, 0.41666667, 0.52777778],
       [0.15833333, 0.04166667, 0.00277778]])
    
    >>fitness_from_prey(x= np.array([1,2,3]), M1=10, M2=10, **dict(b1=1,b2=0.1,r=0, γ=0, a1=1, a2=1, 
                                                        h1=0.5, h2=0.5, 
                                                    α1_of_1=0.05, α2_of_1=0.95, s1=2, s2=2,b0 = 0.05, 
                                                    limited_portions = True))
    array([[0.00416667, 0.04166667, 0.07916667],
       [0.07916667, 0.04166667, 0.00277778]])
    '''
    # set portion size, need to account for x being an array

        
    w_per_capita = per_capita_fitness_from_prey(x,M1,M2, **params)
    try:
        if x > 1:
            repro_exchange = (1-γ)*(1-r) + r*x
            return w_per_capita * repro_exchange
        else:
            return w_per_capita
    except ValueError:
        repro_exchange = np.ones(np.shape(x))
        repro_exchange[x>1] = (1-γ)*(1-r) + r*x[x>1]
        return  w_per_capita * repro_exchange

def conversion_prey(x,b1,b2,limited_portions,**params):
    '''
    Finds the number of predators produced per prey item caught, for big prey and small prey
    @inputs:
    x - pred group size, could be vector
    b1 - number preds produced from big prey if completely eaten
    b2 - number preds produced from small prey if completely eaten
    limited_portions- whether there's a limit of how much food one predator can eat
    params - dic of parameters used by rest of model
    
    @returns
    conversion_big, conversion_small (both floats)

    @examples
    >> conversion_prey(x=2,b1=1,b2=0.1,limited_portions=True,**dict(b0 = 0.1))
    (0.1, 0.05)
    >> conversion_prey(x=[1,2],b1=1,b2=0.1,limited_portions=True,**dict(b0 = 0.1))
    (array([0.1, 0.1]), array([0.1 , 0.05]))
    >> conversion_prey(x=2,b1=1,b2=0.1,limited_portions=False,**dict(b0 = 0.1))
    (0.5, 0.05)
    '''
    if limited_portions == True:
        b0 = params["b0"]
        try:
            conversion_big = b0 if b1/x > b0 else b1/x
            conversion_small = b0 if b2/x > b0 else b2/x 
        except (ValueError,TypeError): # if x is a list (-->TypeError) or an ndarray (-->ValueError)
            x = np.array(x)
            conversion_big = np.ones(np.shape(x))
            conversion_small = conversion_big.copy()
            conversion_big = b1/x
            conversion_big[conversion_big>b0] = b0
            conversion_small = b2/x
            conversion_small[conversion_small>b0] = b0
    else:
        conversion_big = b1/x
        conversion_small = b2/x
    return conversion_big, conversion_small
    
        
        
def fun_fitness_from_big_prey(x, M1, M2, **params):
    '''
    portion of inclusive fitness from big prey type. calls fitness_from_prey
    @inputs:
    x - pred group size
    M1 - big prey pop size
    M2 - small prey pop size
    b1 - big prey  conversion (prey --> pred)
    b2 - small prey conversion (prey --> pred)
    params - dictionary of other parameters, which must at least contain 
             r, γ, a1, a2, h1, h2, α1_of_1, α2_of_1, s1, s2, limited_portions
             (and must also have b0 if limited_portions = True)

    @returns:
    <inclusive fitness from big prey>
    '''
    return fitness_from_prey(x, M1, M2, **params)[0]
def fun_fitness_from_small_prey(x, M1, M2, **params):
    '''
    portion of inclusive fitness from small prey type. calls fitness_from_prey
    @inputs:
    x - pred group size
    M1 - big prey pop size
    M2 - small prey pop size
    b1 - big prey  conversion (prey --> pred)
    b2 - small prey conversion (prey --> pred)
    params - dictionary of other parameters, which must at least contain 
             r, γ, a1, a2, h1, h2, α1_of_1, α2_of_1, s1, s2, limited_portions
             (and must also have b0 if limited_portions = True)

    @returns:
    <inclusive fitness from small prey>

    @example
    >> fun_fitness_from_small_prey(1, 10, 10, **dict(b1=1,b2=0.1,r=0, γ=0, a1=1, a2=1, 
    >>                                    h1=0.5, h2=0.5, α1_of_1=0.05, α2_of_1=0.95, s1=2, s2=2,
                                          limited_portions = False))
    0.15833333333333333
    '''
    return fitness_from_prey(x, M1, M2, **params)[1]


######## non-dimensionalized ###############
# TO-DO: what do I do if 


Overwriting fitness_funs.py


In [46]:
np.matmul(np.array([[1,2,3],[0,2,3],[0,0,3]]),np.transpose(np.array([2,2,2])))

array([12, 10,  6])

In [4]:
np.log(0.95/0.05)

2.9444389791664403

In [170]:
%%writefile group_formation_funs.py
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy.integrate import odeint, solve_ivp
from fun_response_funs import *
from fitness_funs import *

            

def group_formation_model_separate(t, f_of_x_vec,p,M1,M2, params):
    '''
    the full system of balance equations for x = 1,2,3,...,x_max
    @inputs:
    t - time, necessary for running with solve_ivp
    f_of_x_vec - a vector of the number of groups of size 1, 2, 3, ..., x_max (maximum group size)
    p - population size of predators
    M1 - population size of big prey
    M2 - population size of small prey
    params - is a dictionary of the parameters that must contain: 
            b1, b2,r, γ, a1, a2, h1, h2, α1_of_15, α2_of_1, s1, s2, limited_portions, 
            τx, δ, d
            (and b0 if limited_portions = False)
    @ returns
    df_dt for x = 1, 2, ..., xmax
    '''
    x_max = params['x_max']; τx = params['τx']; δ = params['δ']; d = params['d']
    f_of_x_vec = np.append(f_of_x_vec,0) # so can find dfdt at x = x_max
    def f(x):
        return f_of_x_vec[x-1]
    def S(x,y):
        return best_response_fun_given_fitness(x,y,fitnessvec,d)
    def ψ(x):
        f_of_1 = f_of_x_vec[0]
        if x== 1 and f_of_1 >=1:
            return (f_of_1 - 1) * S(2,1)
        elif x <= x_max - 1:
            return f_of_1*S(x+1,1)
        else:
            return 0
    def ϕ(x):
        return x*S(1,x) if x <= x_max else 0
    
    x_max = params['x_max']; τx = params['τx']; δ = params['δ']; d = params['d']
    xvec = np.arange(1,x_max+1,1)
    # it \tau_x > 0make population matrix = birth matrix + death matrix
    fitnessvec = fun_fitness(xvec, M1, M2, **params)
    dfdt_vec = np.zeros(x_max)
    
    for x in xvec:
        if x == 1:
            dfdt = (2*f(2)*ϕ(2) + np.sum([f(y) * ϕ(y) for y in range(3,x_max+1)]) \
                    - sum([f(y-1)*ψ(y-1) for y in range(2,x_max+1)]))/τx
        elif x == 2:
            dfdt = (-f(2)*ϕ(2) - f(2)*ψ(2) + 0.5*f(1)*ψ(1) + f(3)*ϕ(3))/τx
        else:
            dfdt = (-f(x)*ϕ(x) - f(x) * ψ(x) + f(x-1)*ψ(x-1) + f(x+1)*ϕ(x+1))/τx
        
        dfdt_vec[x-1] = dfdt
    return dfdt_vec
    

def fun_1_death(x, τx, δ):
    '''
    The probability of AT LEAST one death in a group of size x over time τ_x
    @inputs:
    x - grp size
    τx - group evolution time constant
    δ - death rate
    params - dictionary of parameters from the ret of the model, not really needed...
    @output:
    float

    @example
    >> fun_1-death(x=1, τx = 0.01, δ = 0.1)
    0.0010000000000000009
    >> funfdgf_1_death(np.array([1,2,3]), 0.01, 0.1)
    array([0.001   , 0.001999, 0.002997])sxz
    
    '''
    return 1 - (1 - δ*τx)**x

def fun_death_y_to_x(x, y, τx, δ, x_max):
    '''
    The probability a group of size y shrinks to a group of size x because y - x individuals die, 
    works for for x < y, y <= x_max

    @inputs:
    x = group size after death, is the shape of what is being returned
    y = original group size, y > x, y <= x_max
    δ = death rate
    x_max = maximum group size
    params = dictionary of other parameters used in the model

    @output:
    float between 0 and 1 (note for τx small, fun_death_y_to_x(x,y,**params) \approx 0 if x < y-1

    @example
    >>fun_death_y_to_x(x=2, y=3, **dict(τx=0.01, δ=0.1, x_max=10))
    0.0029940030000000003
    '''
    if isinstance(y, np.ndarray):
        to_return = np.zeros(y.shape)
        notzero = x<y
        y = y[notzero]
        if isinstance(x, np.ndarray):
            x = x[notzero]
        to_return[notzero] = nchoosek(y,y-x) * (δ*τx)**(y-x)*(1-δ*τx)**x
        return to_return
    else:
        if x < y:
            return nchoosek(y,y-x) * (δ*τx)**(y-x)*(1-δ*τx)**x
            
def nchoosek(n,k):
    '''
    n choose k
    n!/(k!(n-k)!)
    @inputs:
    n and k are integers, but can handle np.arrays
    @returns:
    positive integer (or array if inputs are arrays
    @example
    >> nchoosek(3,1)
    3.0
    >> nchoosek(np.array([3,2]),1)
    array([3.,2.])
    '''
    return sp.special.factorial(n)/(sp.special.factorial(k)*sp.special.factorial(n-k))

def fun_leave_group(x, fitnessvec, x_max, d):
    '''
    The probability an individual leaves a group of size x.
    This is ϕ(x) in the text
    @inputs
    x - current grp size (before leaving)
    fitnessvec = vector of fitnesses for each group size
    x_max - parameter, maximum group size
    d = parameter determining steepness of best response function

    @ example:
    >> fitnessvec = array([0.24166667, 0.45833333, 0.53055556])
    >> fun_leave_group(xvec=[1,2,3], fitnessvec, x_max=3, d=100)
    array([0.5       , 0.03915869, 0.01923075])
    '''
    # deciding between being alone and staying in group of size x
    return best_response_fun_given_fitness(1,x,fitnessvec,d)

def best_response_fun_given_fitness(x,y,fitnessvec, d):
    '''
    Compares W(x) to W(y) to "decide" on group size y or x
    @inputs
    x - potentially new group size
    y - current grp size
    fitnessvec - vector of fitnesses fro x = 1, 2, ..., xmax
    d - steepness, or sensitivity, of best response function
    params - dictionary of params used by the rest of the model 
    @returns:
    float between 0 and 1
    
    '''
    W_of_x = fitnessvec[x-1]
    W_of_y = fitnessvec[y-1]
    return W_of_x**d/(W_of_x**d + W_of_y**d)
    
def best_response_fun(x,y,M1,M2, d, **params):
    '''
    Compares W(x) to W(y) to "decide" on group size y or x
    @inputs
    x - potentially new group size
    y - current grp size
    M1 - big prey pop size
    M2 - small prey pop size
    d - steepness, or sensitivity, of best response function
    params - dictionary of params used by the rest of the model, 
            but must include all params relevant to functional responses and 
            inclusive fitness 
    @returns:
    float between 0 and 1

    @example
    >> best_response_fun(x=2,y=3,M1=10,M2=10, d=100, **dict(b1=1,b2=0.1,r=0, γ=0, a1=1, a2=1, h1=0.5, h2=0.5, 
    >>                                              α1_of_1=0.05, α2_of_1=0.95, s1=2, s2=2) )
    4.4162891945392386e-07
    >> 1 - best_response_fun(x=3,y=2,M1=10,M2=10, d=100, **dict(b1=1,b2=0.1,r=0, γ=0, a1=1, a2=1, h1=0.5, h2=0.5, 
    >>                                              α1_of_1=0.05, α2_of_1=0.95, s1=2, s2=2) )
    4.4162891943422267e-07
    
    '''
    W_of_x = fun_fitness(x,M1,M2, **params)
    W_of_y = fun_fitness(y, M1, M2, **params)
    return W_of_x**d/(W_of_x**d + W_of_y**d)


Overwriting group_formation_funs.py


In [126]:
params = dict(α1_of_1 = 0.05, α2_of_1 = 0.95, s1 = 2, s2 = 2, 
              a1=1,a2=1,h1=0,h2=0, b1 = 1, b2 = 0.1, r = 0, γ = 0,
             limited_portions=False, x_max = 3, δ = 0, τx = 1, d = 10)
fitnessvec = fun_fitness(np.array([1,2,3]), M1=10, M2=10, **params)
print(fitnessvec)

[1.45       2.75       3.18333333]


In [116]:
dfdt_ = group_formation_model_separate(t=1, f_of_x_vec = np.array([8,1,0]),p = 10, 
                                       M1 = 10,M2 = 10, params=params)

In [22]:
t=1; f_of_x_vec = np.array([10,0,0]); p = 10; M1 = 10; M2 = 10;  params=params
x_max = params['x_max']; τx = params['τx']; δ = params['δ']; d = params['d']
def f(x):
    return f_of_x_vec[x-1]
def S(x,y):
    return best_response_fun_given_fitness(x,y,fitnessvec,d)
def ψ(x):
    f_of_1 = f_of_x_vec[0]
    if x== 1 and f_of_1 >=1:
        return (f_of_1 - 1) * S(2,1)
    elif x <= x_max - 1:
        return f_of_1*S(x+1,1)
    else:
        return 0
def ϕ(x):
    return x*S(1,x) if x <= x_max else 0

In [111]:
f_of_x_vec = np.array([8,1,0]);
fitnessvec = fun_fitness(xvec, M1, M2, **params)
dfdt_vec = np.zeros(x_max)
f_of_x_vec = np.append(f_of_x_vec,0)
for x in xvec:
    if x == 1:
        dfdt = (2*f(2)*ϕ(2) + np.sum([f(y) * ϕ(y) for y in range(3,x_max+1)]) \
                - sum([f(y-1)*ψ(y-1) for y in range(2,x_max+1)]))/τx
    elif x == 2:
        dfdt = (-f(2)*ϕ(2) - f(2)*ψ(2) + 0.5*f(1)*ψ(1) + f(3)*ϕ(3))/τx
    else:
        dfdt = -f(x)*ϕ(x) - f(x) * ψ(x) + f(x-1)*ψ(x-1) + f(x+1)*ϕ(x+1)
    
    dfdt_vec[x-1] = dfdt
   # print(dfdt_vec[x-1] )

In [154]:
dfdt_ = group_formation_model_separate(t=1, f_of_x_vec = np.array([8,1,0]),p = 10, 
                                       M1 = 10,M2 = 10, params=params)
print(dfdt_)
xvec = np.array([1,2,3])
print(sum(xvec*dfdt_))

[-63.89743449  19.95332908   7.99692545]
0.0


In [160]:
p_reg = 10;initialstate = np.array([p_reg, *np.zeros(x_max - 1)]);


In [161]:
params = dict(b1 = 1, b2 = 0.1, α1_of_1 = 0.05, α2_of_1 = 0.95, s1 = 2, s2 = 2, 
              a1=1,a2=1, h1=0, h2=0, r = 0, γ = 0,
             limited_portions=False, x_max = 3, δ = 0, τx = 1, d = 100)
p_reg = 50; M1_reg = 100; M2_reg = 100
initialstate = np.array([p_reg, *np.zeros(x_max - 1)]);
t_f = 2
out = solve_ivp(group_formation_model_separate, [0, t_f], initialstate, method="LSODA",
                args=(p_reg, M1_reg, M2_reg, params))

In [164]:
out.y[:,-1]

array([5.28628065e-03, 2.94745315e+00, 1.46999358e+01])

In [163]:
xvec*out.y[:,-1]

array([5.28628065e-03, 5.89490629e+00, 4.40998074e+01])

In [162]:
sum(xvec*out.y[:,-1])

50.00000000000003

In [150]:
p_reg = 50
M1_reg = 100; M2_reg = 100
h_reg = 0
params_reg = dict(b1=1,b2=0.1,a1 = 1, a2 = 1, h1 = h_reg,h2 = h_reg, α1_of_1 = 0.05, α2_of_1 = 0.95, s1 = 2, 
                  s2 = 2, δ=0, τx=0.01, r=0, γ=0, x_max=3, d = 100, limited_portions = False)
#param_vec = [2,3,8]
#param_key = 's2'
x_max = params_reg['x_max']

In [151]:
initialstate = np.array([p_reg, *np.zeros(x_max - 1)]);

t_f = 2
out = solve_ivp(group_formation_model_separate, [0, t_f], initialstate, method="LSODA",
                args=(p_reg, M1_reg, M2_reg, params_reg))

In [152]:
np.round(out.y[:,-1],3)

array([0.   , 2.949, 0.147])

In [153]:
np.sum(np.arange(1,4,1)*np.transpose(out.y)[-1,:])

6.339897382398504

In [65]:
2 + .75*2 + .84*3

6.02

In [72]:
xvec*out.y[:,-1]

array([1.96817898, 1.49618326, 2.51383699])

In [53]:
print(dfdt_)

[-58.92857143  29.46428571   0.        ]


In [54]:
sum(xvec*dfdt_)


0.0

In [17]:
xvec = np.array([1,2,3])
sum(xvec*dfdt_)

dfdt1 = 

-5.0

-58.92857142857143

In [39]:
- f(1) * sum([ψ(y-1) for y in range(2,x_max)])

-58.92857142857143

In [20]:
x_max = params['x_max']; τx = params['τx']; δ = params['δ']; d = params['d']
xvec = np.arange(1,x_max+1,1)
dfdt_vec = np.zeros(x_max)
print(dfdt_vec)

dfdt_1 = f(2)*

[0. 0. 0.]


In [44]:
- f(1) * sum([ψ(y-1) for y in range(2,x_max)])

-58.92857142857143

In [49]:



#return dfdt_vec

-58.92857142857143
29.464285714285715
0.0


In [46]:
dfdt_vec[0]

-63.92857142857143

In [171]:
%%writefile group_formation_funs_vectorized.py
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy.integrate import odeint, solve_ivp
from fun_response_funs import *
from fitness_funs import *

    
def group_formation_model_alt(t, f_of_x_vec,p,M1,M2, pop_process, params):
    '''
    the full system of balance equations for x = 1,2,3,...,x_max
    @inputs:
    t - time, necessary for running with solve_ivp
    f_of_x_vec - a vector of the number of groups of size 1, 2, 3, ..., x_max (maximum group size)
    p - population size of predators
    M1 - population size of big prey
    M2 - population size of small prey
    pop_process = True or False, whether or not to include death rate and birth rate
    params - is a dictionary of the parameters that must contain: 
            b1, b2,r, γ, a1, a2, h1, h2, α1_of_15, α2_of_1, s1, s2, limited_portions, 
            τx, δ, d
            (and b0 if limited_portions = False)
    @ returns
    df_dt for x = 1, 2, ..., xmax
    '''
    x_max = params['x_max']; τx = params['τx']; δ = params['δ']; d = params['d']
    
    xvec = np.arange(1,x_max+1,1)

    # it \tau_x > 0make population matrix = birth matrix + death matrix
    fitnessvec = fun_fitness(xvec, M1, M2, **params)
    if pop_process==True:
        Π = make_population_proc_matrix(xvec, fitnessvec, δ, τx, x_max)
    else: 
        Π = 0
    Ψ = make_group_form_mat(xvec, f_of_x_vec, fitnessvec, x_max,d)
    Φ = make_group_leave_mat(xvec, fitnessvec, x_max, d)
    return np.matmul( Π + Ψ + Φ, np.transpose(f_of_x_vec))/τx

def make_group_leave_mat(xvec, fitnessvec, x_max, d):
    
    ϕ_of_x = best_response_fun_given_fitness(1,xvec,fitnessvec,d)*xvec
    ϕ_of_x[0] = 0 # can't leave a group of 1
    
    first_row_and_upper_diag = ϕ_of_x[1:].copy()
    first_row_and_upper_diag[0] = 2*first_row_and_upper_diag[0] # if group of size 2 --> 1, 
                                                                # produces 2 solitaries
    diag_mat = np.diag(- ϕ_of_x) # fewer groups of size x if grps of size x --> x - 1
    upper_diag_mat = np.diag(first_row_and_upper_diag,k=1) # more grps of size x if grps of size x +1 --> x
    group_shrink_mat = diag_mat + upper_diag_mat
    group_shrink_mat[0,1:] = first_row_and_upper_diag # more solitaries when individuals leave groups

    return group_shrink_mat
def make_group_form_mat(xvec, f_of_x_vec, fitnessvec, x_max,d):
    '''
    example:
    >>params_reg = dict(b1=1,b2=0.1,r=0, γ=0, a1=1, a2=1, h1=0.5, h2=0.5, 
                                                    α1_of_1=0.05, α2_of_1=0.95, s1=2, s2=2, d = 20,
                                                    limited_portions = False)
    >>f_of_x_vec=[1,1,1]
    >> make_group_form_mat(x_vec=np.array([1,2,3]), f_of_x_vec, M1=10, M2=10, x_max=3,**params_reg)
    array([[ 0.        ,  0.        ,  0.        ],
       [ 0.        , -0.99999985,  0.        ],
       [ 0.        ,  0.99999985,  0.        ]])
    '''
    # alterations
    f_of_1_vec = np.full(x_max,f_of_x_vec[0])
    f_of_1_vec[0] = f_of_1_vec[0] - 1
    xvec = xvec[:-1] # get rid of xmax
    
    join_grp_vec = np.zeros(x_max)
    
    best_response_vec = best_response_fun_given_fitness(xvec+1,1,fitnessvec,d) #S(x+1,1)
    join_grp_vec[:-1] = best_response_vec * f_of_1_vec[:-1]

    # make matrix
    diag_mat = np.diag(- join_grp_vec) # loss from class x as x --> x+1
    lower_diag_mat = np.diag(join_grp_vec[:-1],k=-1) # gain to class x+1 as x --> x+1
    group_form_mat = diag_mat + lower_diag_mat
    group_form_mat[-1, -1] = 0 # can't grow once at x_max
    group_form_mat[1,0] = 0.5*group_form_mat[1,0] # individuals forming pairs, have to multiply by 1/2
    return group_form_mat
                         
def make_population_proc_matrix(xvec, fitnessvec, δ, τx, x_max):
    '''
    make the population processes matrix Π = τx Π_W  + Π_D
    @inputs
    xvec = [1,2,..., x_max]
    fitnessvec = [\bar{w}(1), \bar{w}(2), ..., \bar{x_max}(1)] vector of per capita fitnesses
    δ = death rate
    τx = group time scale
    x_max = max grp size

    @returns
    np.ndarray that is x_max x x_max

    @example:
    >> params_reg = dict(b1=1,b2=0.1,r=0,γ=0, a1 = 1, a2 = 1, h1 = 0.5, h2 = 0.5, 
                  α1_of_1 = 0.05, α2_of_1 = 0.95, s1 = 2, s2 = 2, d = 100, limited_portions = False)
    >> xvec = np.array([1,2,3]); M1 = 10; M2 = 10; x_max = 3
    >> fitnessvec = fun_fitness(xvec, M1, M2, **params_reg) # this is array([0.24166667, 0.45833333, 0.53055556])
    >> make_population_proc_matrix(xvec, fitnessvec, δ, τx, x_max)
    array([[ 0.00141667,  0.01116467,  0.01591966],
       [ 0.        , -0.001999  ,  0.002994  ],
       [ 0.        ,  0.        , -0.002997  ]])
    '''
    Π_D = make_death_trans_matrix(xvec, δ, τx, x_max)

    # birth matrix
    Π_W = np.zeros((x_max,x_max))
    Π_W[0,:] = fitnessvec*xvec

    # population matrix
    Π = τx * Π_W + Π_D
    return Π
    
def make_death_trans_matrix(xvec, δ, τx, x_max):
    '''
    makes the death transition matrix
    the diagonal row is D(x)
    above the diagonal, entries are D(i,j), for i the row and j the column

    @inputs:
    xvec - vector 1, 2, ..., x_max
    δ = death rate
    τx = timescale of group dynamics
    x_max = max group size

    @output: an x-max x x_max numpy array

    @example 
    >> make_death_trans_matrix(xvec = np.array([1,2,3]), δ=0.1, τx=0.01, x_max=3)
    array([[-1.000000e-03,  1.998000e-03,  2.997000e-06],
       [ 0.000000e+00, -1.999000e-03,  2.994003e-03],
       [ 0.000000e+00,  0.000000e+00, -2.997001e-03]])
    '''
    # number of groups of size x decreases from deaths in group of size x
    Pi_D = np.zeros((x_max,x_max))
    D_of_x = fun_1_death(xvec, τx, δ)
    np.fill_diagonal(Pi_D, -D_of_x)

    # number of groups of size x increases if a larger group of size y has y-x deaths
    y_mat = np.full((x_max,x_max),xvec)
    x_mat = y_mat.copy().transpose()
    D_transition = fun_death_y_to_x(x=x_mat, y=y_mat, τx=τx, δ=δ, x_max=x_max)
    Pi_D = Pi_D + D_transition
    return Pi_D
def fun_1_death(x, τx, δ):
    '''
    The probability of AT LEAST one death in a group of size x over time τ_x
    @inputs:
    x - grp size
    τx - group evolution time constant
    δ - death rate
    params - dictionary of parameters from the ret of the model, not really needed...
    @output:
    float

    @example
    >> fun_1-death(x=1, τx = 0.01, δ = 0.1)
    0.0010000000000000009
    >> funfdgf_1_death(np.array([1,2,3]), 0.01, 0.1)
    array([0.001   , 0.001999, 0.002997])sxz
    
    '''
    return 1 - (1 - δ*τx)**x

def fun_death_y_to_x(x, y, τx, δ, x_max):
    '''
    The probability a group of size y shrinks to a group of size x because y - x individuals die, 
    works for for x < y, y <= x_max

    @inputs:
    x = group size after death, is the shape of what is being returned
    y = original group size, y > x, y <= x_max
    δ = death rate
    x_max = maximum group size
    params = dictionary of other parameters used in the model

    @output:
    float between 0 and 1 (note for τx small, fun_death_y_to_x(x,y,**params) \approx 0 if x < y-1

    @example
    >>fun_death_y_to_x(x=2, y=3, **dict(τx=0.01, δ=0.1, x_max=10))
    0.0029940030000000003
    '''
    if isinstance(y, np.ndarray):
        to_return = np.zeros(y.shape)
        notzero = x<y
        y = y[notzero]
        if isinstance(x, np.ndarray):
            x = x[notzero]
        to_return[notzero] = nchoosek(y,y-x) * (δ*τx)**(y-x)*(1-δ*τx)**x
        return to_return
    else:
        if x < y:
            return nchoosek(y,y-x) * (δ*τx)**(y-x)*(1-δ*τx)**x
            
def nchoosek(n,k):
    '''
    n choose k
    n!/(k!(n-k)!)
    @inputs:
    n and k are integers, but can handle np.arrays
    @returns:
    positive integer (or array if inputs are arrays
    @example
    >> nchoosek(3,1)
    3.0
    >> nchoosek(np.array([3,2]),1)
    array([3.,2.])
    '''
    return sp.special.factorial(n)/(sp.special.factorial(k)*sp.special.factorial(n-k))

def fun_leave_group(x, fitnessvec, x_max, d):
    '''
    The probability an individual leaves a group of size x.
    This is ϕ(x) in the text
    @inputs
    x - current grp size (before leaving)
    fitnessvec = vector of fitnesses for each group size
    x_max - parameter, maximum group size
    d = parameter determining steepness of best response function

    @ example:
    >> fitnessvec = array([0.24166667, 0.45833333, 0.53055556])
    >> fun_leave_group(xvec=[1,2,3], fitnessvec, x_max=3, d=100)
    array([0.5       , 0.03915869, 0.01923075])
    '''
    # deciding between being alone and staying in group of size x
    return best_response_fun_given_fitness(1,x,fitnessvec,d)

def best_response_fun_given_fitness(x,y,fitnessvec, d):
    '''
    Compares W(x) to W(y) to "decide" on group size y or x
    @inputs
    x - potentially new group size
    y - current grp size
    fitnessvec - vector of fitnesses fro x = 1, 2, ..., xmax
    d - steepness, or sensitivity, of best response function
    params - dictionary of params used by the rest of the model 
    @returns:
    float between 0 and 1
    
    '''
    W_of_x = fitnessvec[x-1]
    W_of_y = fitnessvec[y-1]
    return W_of_x**d/(W_of_x**d + W_of_y**d)
    
def best_response_fun(x,y,M1,M2, d, **params):
    '''
    Compares W(x) to W(y) to "decide" on group size y or x
    @inputs
    x - potentially new group size
    y - current grp size
    M1 - big prey pop size
    M2 - small prey pop size
    d - steepness, or sensitivity, of best response function
    params - dictionary of params used by the rest of the model, 
            but must include all params relevant to functional responses and 
            inclusive fitness 
    @returns:
    float between 0 and 1

    @example
    >> best_response_fun(x=2,y=3,M1=10,M2=10, d=100, **dict(b1=1,b2=0.1,r=0, γ=0, a1=1, a2=1, h1=0.5, h2=0.5, 
    >>                                              α1_of_1=0.05, α2_of_1=0.95, s1=2, s2=2) )
    4.4162891945392386e-07
    >> 1 - best_response_fun(x=3,y=2,M1=10,M2=10, d=100, **dict(b1=1,b2=0.1,r=0, γ=0, a1=1, a2=1, h1=0.5, h2=0.5, 
    >>                                              α1_of_1=0.05, α2_of_1=0.95, s1=2, s2=2) )
    4.4162891943422267e-07
    
    '''
    W_of_x = fun_fitness(x,M1,M2, **params)
    W_of_y = fun_fitness(y, M1, M2, **params)
    return W_of_x**d/(W_of_x**d + W_of_y**d)


Overwriting group_formation_funs_vectorized.py


In [43]:
best_response_fun_given_fitness?

Signature: best_response_fun_given_fitness(x, y, fitnessvec, d)
Docstring:
Compares W(x) to W(y) to "decide" on group size y or x
@inputs
x - potentially new group size
y - current grp size
fitnessvec - vector of fitnesses fro x = 1, 2, ..., xmax
d - steepness, or sensitivity, of best response function
params - dictionary of params used by the rest of the model 
@returns:
float between 0 and 1
File:      /var/folders/tp/jbfyy0qd62910dnqkytn6dqc0000gn/T/ipykernel_35654/505729230.py
Type:      function

In [40]:
make_group_leave_mat(xvec=np.array([1,2,3]), fitnessvec = np.array([0.3,0.1,0.1]), x_max=3, d=100)

array([[ 0.,  2.,  1.],
       [ 0., -1.,  1.],
       [ 0.,  0., -1.]])

In [172]:
%%writefile group_formation_measures.py
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy.integrate import odeint, solve_ivp
from fun_response_funs import *
from fitness_funs import *
from group_formation_funs import *


def mean_group_size_membership(f_of_x_vec,p,x_max):
    '''
    average group size any individual is in
    this is not the same as the average group size
    '''
    xsquared = np.arange(1,x_max+1,1)**2
    total = sum(xsquared*f_of_x_vec)
    return total/p

def mean_group_size(f_of_x_vec,x_max):
    '''
    the average grp sizes
    '''
    total = 0
    num_grps = 0
    total = sum(np.arange(1,x_max+1,1)*np.array(f_of_x_vec))
    num_grps = sum(f_of_x_vec)
    return total/num_grps


def fun_num_groups(f_of_x_vec,p,x_max):
    total = 0
    for x in range(1,x_max+1):
        total += fun_f_of_x(x, f_of_x_vec, p, x_max,**dict())
    return total


def check_at_equilibrium(f_of_x_vec, p, M1, M2,  pop_process, **params):
    '''
    checks that dfdt = 0
    @input:
    f_of_x_vec = (f(1), f(2), ..., f(x_max))
    p, M1, M2 = pred, big prey, and small prey pop sizes, respectively
    pop_process = True or False, whether to include death rates and birth rates
    params = dic of params
    @returns
    1 if at equilibrium, 0 otherwise
    '''
    t = 1 # this doesn't matter
    dfdt_ = group_formation_model_separate(t, f_of_x_vec,p,M1,M2, params)
    at_equilibrium = np.abs(dfdt_) > 1e-10
    if sum(at_equilibrium) > 0: # at least one df(x)/dt is not zero
        return dfdt_, 0
    else:
        return dfdt_, 1

Overwriting group_formation_measures.py


In [168]:
group_formation_model_separate?

Signature: group_formation_model_separate(t, f_of_x_vec, p, M1, M2, params)
Docstring:
the full system of balance equations for x = 1,2,3,...,x_max
@inputs:
t - time, necessary for running with solve_ivp
f_of_x_vec - a vector of the number of groups of size 1, 2, 3, ..., x_max (maximum group size)
p - population size of predators
M1 - population size of big prey
M2 - population size of small prey
params - is a dictionary of the parameters that must contain: 
        b1, b2,r, γ, a1, a2, h1, h2, α1_of_15, α2_of_1, s1, s2, limited_portions, 
        τx, δ, d
        (and b0 if limited_portions = False)
@ returns
df_dt for x = 1, 2, ..., xmax
File:      /var/folders/tp/jbfyy0qd62910dnqkytn6dqc0000gn/T/ipykernel_65876/4228432376.py
Type:      function

In [10]:
sum(np.abs([0.0001, 0]) < 1e-10)

1

In [226]:
#%%writefile group_formation_funs.py
'''
this is retired
'''
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy.integrate import odeint, solve_ivp
from fun_response_funs import *
from fitness_funs import *


def group_formation_model(t, f_of_x_vec,p,M1,M2,params):
    '''
    the full system of balance equations for x = 2,3,...,x_max
    @inputs:
    t - time, necessary for running with solve_ivp
    f_of_x_vec - a vector of the number of groups of size 1, 2, 3, ..., x_max (maximum group size)
    p - population size of predators
    M1 - population size of big prey
    M2 - population size of small prey
    params - is a dictionary of the parameters that must contain: 
            b1, b2,r, γ, a1, a2, h1, h2, α1_of_15, α2_of_1, s1, s2, limited_portions, 
            τx, δ, d
            (and b0 if limited_portions = False)
    '''
    x_max = params["x_max"]
    dfdt = [fun_dfdt(f_of_x_vec, x, p, M1, M2, **params) for x in range(2,x_max+1)]
    #for x in range(2,x_max+1):
    #    dfdt[x-2] = fun_dfdt(f_of_x_vec, x, p, M1, M2, **params)
    return dfdt

def group_formation_model_alt(t, f_of_x_vec,p,M1,M2, τx, δ, x_max, params):
    '''
    THIS IS BEING CONSTRUCTED
    I THINK I SHOULD HAVE SOMETHING THE TRANSITION MATRICES FIRST AS MUCH AS POSSIBLE BECAUSE IN PART THEY STAY THE SAME
    the full system of balance equations for x = 2,3,...,x_max
    @inputs:
    t - time, necessary for running with solve_ivp
    f_of_x_vec - a vector of the number of groups of size 1, 2, 3, ..., x_max (maximum group size)
    p - population size of predators
    M1 - population size of big prey
    M2 - population size of small prey
    params - is a dictionary of the parameters that must contain: 
            b1, b2,r, γ, a1, a2, h1, h2, α1_of_15, α2_of_1, s1, s2, limited_portions, 
            τx, δ, d
            (and b0 if limited_portions = False)
    '''
    def f(y):
        return f_of_x_vec[y-1] if y <= x_max else 0
    def D_tot(y):
        '''
        probability there is at least one death in group of size y
        '''
        return fun_1_death(y, τx, δ, **params)
    def D(z1,z2):
        # probability group of size z2 shrinks to group of size z1 for z1 \leq x_max  
        return fun_death_y_to_x(z1,z2, τx, δ, x_max, **params) if z1<z2 else 0
    def ϕ(y):
        # probability individual leaves group of size y for y <= x_max
        return fun_leave_group(y, M1, M2, x_max, **params) if y in range(2,x_max+1) else 0
    def ψ(y):
        return fun_join_group(y, M1, M2, x_max, **params) if y in range(1,x_max) else 0
    
    x_max = params["x_max"]
    dfdt = np.zeros(f_of_x_vec.shape)

    # births
    births = τx*sum([x*f(x)*per_capita_fitness_from_prey(x, M1, M2,**params) for x in range(1,x_max+1)])
    dfdt[0] = dfdt[0] + births

    #deaths - transition from one size
    death_transition_matrix = np.array([[D(x,y) for y in range(1,x_max+1)] for x in range(0,x_max)]) 
        # rows are size it shrinks to, columns are original size
    rate_size_x_after_deaths = np.matmul(death_transition_matrix,f_of_x_vec) # but 0th element 
                                                                            # is the number of size 0
    dfdt[1:] += rate_size_x_after_deaths[1:]
    
    # deaths - transition out of group of size x (could also do some along columns
    deaths_loss = np.array([D_tot(x) for x in range(1,x_max+1)])
    # TO DO: CHECK THIS IS THE SAME AS SUMMING ALONG COLUMNS OF death_transition_matrix
    rate_deaths_loss = f_of_x_vec*deaths_loss
    dfdt += rate_deaths_loss

    # groups shrink from individuals leaving
    
    
    return dfdt



def fun_dfdt(f_of_x_vec, x, p, M1, M2,  τx, δ, x_max, **params):
    '''
    fun_dfdt: This calculates the change in the distribution f(x) wrt time for x >= 2
        τx df/dt = -xf(x)ϕ(x) - f(1) f(x) ψ(x) - f(x) D(x) 
                + f(x+1)ϕ(x+1) + sum_{y=x+1}^{x_max} f(y) D(y)
        but slightly different for x = 2, x = x_max
    f(x) is the number of groups of size x
    @inputs
    f_of_x_vec - vector of f(x) for x = 1, 2, .., x_max.
    x - grp size, must be >= 2
    p - pred pop size
    M1 - big prey pop size
    M2 - small prey pop size
    τx - group size change time scale
    δ - pred death rate
    x_max - max grp size
    params - dictionary of params used by rest of model

    @returns
    float dfdt

    @example
    >> fun_dfdt(f_of_x_vec = [0,0], x=2, p=100, M1=0, M2=100, **dict(τx= 0.01, δ=0, x_max=3, 
    >>                                                          b1=1,b2=0.1,r=0, γ=0, a1=1, 
    >>                                                          a2=1, h1=0.5, h2=0.5, 
    >>                                              α1_of_1=0.05, α2_of_1=0.95, s1=2, s2=2, d = 100,
                                                        ))
    6.209899910586194e-26
    
    '''
    # get f(x), f(1), and f(x+1)
    
    def f(y):
        if y <= x_max:
            return f_of_x_vec[y-1]
        else:
            return 0
    def D_tot(y):
        '''
        probability there is at least one death in group of size y
        '''
        return fun_1_death(y, τx, δ, **params)
    def D(z1,z2):
        # probability group of size z2 shrinks to group of size z1 for z1 \leq x_max  
        return fun_death_y_to_x(z1,z2, τx, δ, x_max, **params)
    def ϕ(y):
        # probability individual leaves group of size y for y <= x_max
        return fun_leave_group(y, M1, M2, x_max, **params) 
    def ψ(y):
        return fun_join_group(y, M1, M2, x_max, **params)
    


    # if x = 2, τ_x df_dx, group formation is different. it is 1/2 f(1)^2 ψ(1) instead of f(x-1)f(1) ψ(x-1)
    # if x = x_max, the group cannot grow larger and there are no larger groups that can shrink to be that size
    
    if x == 1:
        births = τx*sum([x*f(x)*per_capita_fitness_from_prey(x, M1, M2,**params) for x in range(1,x_max+1)])
        individuals_leave_grps = sum([x*f(x)*ϕ(x) for x in range (2,x_max+1)])
        larger_grp_shrinks = 2*f(2)*ϕ(2)
        solitaried_die = f(1)*
    if x >= 2
        individual_leaves = x*f(x) * ϕ(x)
        grows_to_larger_group = f(1)*f(x) * ψ(x) if x<x_max else 0
        death_in_group = f(x) * D_tot(x)
        if x == 2:
            #join_smaller_grp = f(1)*f(x-1)*ψ(x-1)
            join_smaller_grp = (1/2)*f(1)*(f(1)-1)*ψ(1)
        else:
            join_smaller_grp = f(1)*f(x-1)*ψ(x-1)
        larger_grp_shrinks = (x+1)*f(x+1)*ϕ(x+1) if x < x_max else 0
        death_in_larger_grp = sum([f(y)*D(x,y) for y in range(x+1,x_max+1)]) if x < x_max else 0
        
    #dfdt_times_taux = x*f_of_x*fun_leave_group(x) - 
    
    
        return 1/τx * (-individual_leaves - grows_to_larger_group - death_in_group  
                  + join_smaller_grp + larger_grp_shrinks + death_in_larger_grp)

SyntaxError: invalid syntax (1913556089.py, line 152)